In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from torchvision import transforms
from PIL import Image 

def load_and_preprocess_img(
    path, new_size, crop, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
):
    img = Image.open(path)
    compose = transforms.Compose(
        [
            transforms.Resize(new_size),
            transforms.CenterCrop(crop),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ]
    )
    img = compose(img)
    img = img.unsqueeze(0).permute((0, 2, 3, 1))
    return img.numpy()

In [3]:
import ivy
ivy.set_backend("torch")

# Load image
this_dir = "/models/images/cat.jpg" 
img = ivy.asarray(load_and_preprocess_img(this_dir, 256, 224))



test

In [4]:
# conv = ivy.Conv2D(3, 128, [1,1], 5, 4)
# display(img.shape)

# output = conv(img)
# display(output.shape)

# conv = ivy.Conv2D(128 , 256, [1,1], 5, 4)
# output = conv(output)
# display(output.shape)


procedural model execution for debugging start

In [5]:
# ConvBlock

def ConvBlock(in_chaivyels, out_chaivyels, kernel_size, stride, padding, x):
    conv = ivy.Conv2D(in_chaivyels, out_chaivyels, kernel_size, stride, padding)
    bn = ivy.BatchNorm2D(out_chaivyels)
    activation = ivy.ReLU()

    x = conv(x)
    x = bn(x)
    x = activation(x)
    return x

In [6]:
# _ConvBlock

class _ConvBlock(ivy.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        super(_ConvBlock, self).__init__()

        self.conv = ivy.Conv2D(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = ivy.BatchNorm2D(out_channels)
        self.activation = ivy.ReLU()

    def _forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

In [7]:
# Inception Block

def InceptionBlock(in_channels, num1x1, num3x3_reduce, num3x3, num5x5_reduce, num5x5, pool_proj, x):
    block1 = ivy.Sequential(_ConvBlock(in_channels, num1x1, kernel_size=[1, 1], stride=1, padding=0))

    block2 = ivy.Sequential(_ConvBlock(in_channels, num3x3_reduce, kernel_size=[1, 1], stride=1, padding=0),
                            _ConvBlock(num3x3_reduce, num3x3, kernel_size=[3, 3], stride=1, padding=1))
    
    block3 = ivy.Sequential(_ConvBlock(in_channels, num5x5_reduce, kernel_size=[1, 1], stride=1, padding=0),
                            _ConvBlock(num5x5_reduce, num5x5, kernel_size=[5, 5], stride=1, padding=2))
    
    block4 = ivy.Sequential(_ConvBlock(in_channels, pool_proj, kernel_size=[1, 1], stride=1, padding=0))

    block1 = block1(x)
    block2 = block2(x)
    block3 = block3(x)
    block4 = block4(x)
    
    return ivy.concat([block1, block2, block3, block4], axis=3)

In [8]:
# Auxiliary Block

def AuxiliaryBlock(in_channels, num_classes, x):
    pool = ivy.AdaptiveAvgPool2d((4, 4))
    conv = ivy.Conv2D(in_channels, 128, [1,1], 1, 0)
    activation = ivy.ReLU()
    fc1 = ivy.Linear(2048, 1024)
    dropout = ivy.Dropout(0.7)
    fc2 = ivy.Linear(1024, num_classes)

    
    out = pool(x)
    display(out.shape)
    out = conv(out)
    display(out.shape)
    out = activation(out)
    display(out.shape)
    out = ivy.flatten(out)
    display(out.shape)
    out = fc1(out)
    display(out.shape)
    out = activation(out)
    display(out.shape)
    out = dropout(out)
    display(out.shape)
    out = fc2(out)
    display(out.shape)
    return out

In [9]:
# MaxPool2D

def MaxPool2D(kernel_size, stride, padding, x):
    pool = ivy.MaxPool2D(kernel_size, stride, padding)
    x = pool(x)
    return x

In [10]:
# AdaptiveAvgPool2D

def AdaptiveAvgPool2D(output_size, x):
    avg_pool = ivy.AdaptiveAvgPool2d(output_size)
    x = avg_pool(x)
    return x

In [35]:
# Dropout

def Dropout(prob, x):
    # prob: The probability of zeroing out each array element.
    dropout = ivy.Dropout(prob)
    x = dropout(x)
    return x

In [12]:
# FC

def FC(input_channels, output_channels, x):
    linear = ivy.Linear(input_channels, output_channels)
    x = linear(x)
    return x

In [13]:
# Flatten

def Flatten(x):
    x = ivy.flatten(x)
    return x

In [14]:
# GoogleNet V1

num_classes = 1000
display(img.shape)
out = ConvBlock(3, 64, [7,7], 2, 3, img)
display(out.shape)

ivy.Shape(1, 224, 224, 3)

ivy.Shape(1, 112, 112, 64)

In [15]:
out = MaxPool2D([3,3], 2, 1, out)
display(out.shape)


ivy.Shape(1, 56, 56, 64)

In [16]:
out = ConvBlock(64, 64, [1,1], 1, 0, out)
display(out.shape)


ivy.Shape(1, 56, 56, 64)

In [17]:
out = ConvBlock(64, 192, [3,3], 1, 1, out)
display(out.shape)


ivy.Shape(1, 56, 56, 192)

In [18]:
out = MaxPool2D(3, 2, 1, out)
display(out.shape)


ivy.Shape(1, 28, 28, 192)

In [19]:
out = InceptionBlock(192, 64, 96, 128, 16, 32, 32, out)
display(out.shape)


ivy.Shape(1, 28, 28, 256)

In [20]:
out = InceptionBlock(256, 128, 128, 192, 32, 96, 64, out)
display(out.shape)


ivy.Shape(1, 28, 28, 480)

In [21]:
out = MaxPool2D(3, 2, 1, out)
display(out.shape)


ivy.Shape(1, 14, 14, 480)

In [22]:
out_1 = InceptionBlock(480, 192, 96, 208, 16, 48, 64, out)
display(out_1.shape)

ivy.Shape(1, 14, 14, 512)

In [23]:
in_channels1=512
num_classes=1000
pool = ivy.AvgPool2D((5, 5), 3, 0) # ivy.Shape(1, 4, 4, 512)
conv = ivy.Conv2D(in_channels1, 128, [1,1], 1, 0)
activation = ivy.ReLU()
fc1 = ivy.Linear(2048, 1024)
dropout = ivy.Dropout(0.7)
fc2 = ivy.Linear(1024, num_classes)

display(out_1.shape)
out_3 = pool(out_1)
display(out_3.shape)
out = conv(out_3)
out = activation(out)
out = ivy.flatten(out)
display(out.shape)
out = fc1(out)
display(out.shape)
out = activation(out)
display(out.shape)
out = dropout(out)
display(out.shape)
out = fc2(out)
display(out.shape)
aux1 = out
display(aux1.shape)

ivy.Shape(1, 14, 14, 512)

ivy.Shape(1, 4, 4, 512)

ivy.Shape(2048)

ivy.Shape(1024)

ivy.Shape(1024)

ivy.Shape(1024)

ivy.Shape(1000)

ivy.Shape(1000)

In [24]:
out = InceptionBlock(512, 160, 112, 224, 24, 64, 64, out_1)
display(out.shape)

ivy.Shape(1, 14, 14, 512)

In [25]:
out = InceptionBlock(512, 128, 128, 256, 24, 64, 64, out)
display(out.shape)
out_5 = InceptionBlock(512, 112, 144, 288, 32, 64, 64, out)
display(out_5.shape)

ivy.Shape(1, 14, 14, 512)

ivy.Shape(1, 14, 14, 528)

In [26]:
in_channels1=528
num_classes=1000
pool = ivy.AvgPool2D((5, 5), 3, 0)
conv = ivy.Conv2D(in_channels1, 128, [1,1], 1, 0)
activation = ivy.ReLU()
fc1 = ivy.Linear(2048, 1024)
dropout = ivy.Dropout(0.7)
fc2 = ivy.Linear(1024, num_classes)

display(out_5.shape)
out_3 = pool(out_5)
display(out_3.shape)
out = conv(out_3)
out = activation(out)
out = ivy.flatten(out)
display(out.shape)
out = fc1(out)
display(out.shape)
out = activation(out)
display(out.shape)
out = dropout(out)
display(out.shape)
out = fc2(out)
display(out.shape)
aux2 = out
display(aux2.shape)

ivy.Shape(1, 14, 14, 528)

ivy.Shape(1, 4, 4, 528)

ivy.Shape(2048)

ivy.Shape(1024)

ivy.Shape(1024)

ivy.Shape(1024)

ivy.Shape(1000)

ivy.Shape(1000)

In [27]:
out = InceptionBlock(528, 256, 160, 320, 32, 128, 128, out_5)
display(out.shape)
out_6 = MaxPool2D(3, 2, 1, out)
display(out_6.shape)

ivy.Shape(1, 14, 14, 832)

ivy.Shape(1, 7, 7, 832)

In [28]:
out = InceptionBlock(832, 256, 160, 320, 32, 128, 128, out_6)
display(out.shape)
out_7 = InceptionBlock(832, 384, 192, 384, 48, 128,128, out)
display(out_7.shape)


ivy.Shape(1, 7, 7, 832)

ivy.Shape(1, 7, 7, 1024)

In [36]:
pool7 = ivy.AvgPool2D((7,7), 1, 0)
out = pool7(out_7)
display(out.shape)

out = Flatten(out)
display(out.shape)
display(out)
out_8 = Dropout(0.4, out)
display(out_8)

ivy.Shape(1, 1, 1, 1024)

ivy.Shape(1024)

ivy.array([  0.        ,  76.70142365,  38.29629517, ..., 103.5130539 ,
       112.5506134 , 248.4332733 ])

ivy.array([  0.        ,   0.        ,  63.82715607, ..., 172.521759  ,
       187.5843506 ,   0.        ])

In [38]:
out = FC(1024, num_classes, out_8)

display(out.shape, aux1.shape, aux2.shape)

ivy.Shape(1000)

ivy.Shape(1000)

ivy.Shape(1000)